## Jumanのインストール

In [24]:
import numpy as np
import csv
import glob
import pickle

#*******************************************************************************
#                                                                              *
#   単語補正                                                                   *
#                                                                              *
#*******************************************************************************
def modification(word) :
    modified = [word]
    return modified

#*******************************************************************************
#                                                                              *
#   品詞分解                                                                   *
#                                                                              *
#*******************************************************************************
def decomposition(file, jumanpp) :
    f=open(file, 'r')
    df1 = csv.reader(f)
    data = [ v for v in df1]
    print('number of rows :', len(data))

    parts = []
    for i in range(len(data)) :
        data[i][0] = data[i][0].replace(' ', '')
        if len(data[i][0].encode('utf-8')) <= 4096 :
            result = jumanpp.analysis(data[i][0])
        else :
            print(i, ' skip')
            continue
        for mrph in result.mrph_list():
            parts += modification(mrph.midasi)
        if i % 5000 == 0 :
            print(i)
    return parts


In [56]:
# coding: utf-8
from __future__ import unicode_literals
from pyknp import Juman
jumanpp = Juman()
file_list=glob.glob('tweet/tweet2020-11-28.txt')
file_list.sort()
print(len(file_list))

parts_list = []
for j in range(len(file_list)) :
    print(file_list[j])
    parts_list += decomposition(file_list[j], jumanpp)

with open('parts_list.pickle', 'wb') as f :    
    pickle.dump(parts_list , f)  


1
tweet/tweet2020-11-28.txt
number of rows : 132
0


In [83]:
# coding: utf-8
from __future__ import unicode_literals
import pickle
def genarate_npy(source_csv ,list_corpus) :
    with codecs.open(source_csv, "rb") as f :
        df2 = pickle.load(f)

        mat = [ line for line in df2]
        j=0
        #補正
        for i in range(0,len(mat)):
            if len(mat[i]) !=  0 :
                if mat[i] != '' :

                    if mat[i] != '@' and mat[i] != 'EOS' and mat[i] != ':' and mat[i] != '\\' :
                        if mat[i] == 'REQ' and mat[i+1] == ':' : #デリミタ「REQ:」対応
                            list_corpus.append('REQREQ')
                        elif mat[i] == 'RES' and mat[i+1] == ':' : #デリミタ「RES:」対応
                            list_corpus.append('RESRES')
                        else :
                            list_corpus.append(mat[i])
                        if i % 1000000 == 0 :
                            print(i,list_corpus[j])
                        j += 1
    print(len(list_corpus))
    del mat
    return 

#**********************************************************************************
#                                                                                 *
#  メイン処理                                                                     *
#                                                                                 *
#**********************************************************************************
if __name__ == '__main__':
    import numpy as np
    import csv
    import glob
    import re
    import pickle
    import codecs

    file_list = glob.glob('juman/*')
    print(len(file_list))

    n_words = 0
    for j in range(0,len(file_list)) :

        print(file_list[j])
        generated_list=[]
        genarate_npy(file_list[j],generated_list)
        #コーパスリストセーブ
        with open('list_corpus/list_corpus'+str(j)+'.pickle', 'wb') as g :
            pickle.dump(generated_list , g)
        n_words += len(generated_list)
        print(n_words)
        del generated_list


1
juman/parts_list.pickle
0 REQREQ
2863
2863


In [95]:
# coding: utf-8
def generate_mat() :

    file_list = glob.glob('list_corpus/*')
    print('ファイル数 =',len(file_list))
    mat=[]
    for i in range (0,len(file_list)) :
        with open(file_list[i],'rb') as f :
            generated_list=pickle.load(f)         #生成リストロード
            mat.extend(generated_list)
            print(i)
            del generated_list


    mat.append('REQREQ')

    print("len(mat)",len(mat))
    
    words = sorted(list(set(mat)))
    cnt = np.zeros(len(words))

    print('total words:', len(words))
    word_indices = dict((w, i) for i, w in enumerate(words))    #単語をキーにインデックス検索
    indices_word = dict((i, w) for i, w in enumerate(words))    #インデックスをキーに単語を検索

    #単語の出現数をカウント
    for j in range (0,len(mat)):
        cnt[word_indices[mat[j]]] += 1

    #出現頻度の少ない単語を「UNK」で置き換え
    words_unk = []                                #未知語一覧

    for k in range(0,len(words)):
        if cnt[k] <= 0 :
            words_unk.append(words[k])
            words[k] = 'UNK'

    print('words_unk:',len(words_unk))                   # words_unkはunkに変換された単語のリスト

    #低頻度単語をUNKに置き換えたので、辞書作り直し
    words = list(set(words))
    words.append('\t')                                   #０パディング対策。インデックス０用キャラクタを追加
    words = sorted(words)
    print('new total words:', len(words))
    word_indices = dict((w, i) for i, w in enumerate(words))    #単語をキーにインデックス検索
    indices_word = dict((i, w) for i, w in enumerate(words))    #インデックスをキーに単語を検索

    #単語インデックス配列作成
    mat_urtext = np.zeros((len(mat),1),dtype=int)
    for i in range(0,len(mat)):
        if mat[i] in word_indices :           #出現頻度の低い単語のインデックスをunkのそれに置き換え
            mat_urtext[i,0] = word_indices[mat[i]]
        else:
            mat_urtext[i,0] = word_indices['UNK']

    print(mat_urtext.shape)

    #作成した辞書をセーブ
    with open('word_indices.pickle', 'wb') as f :
        pickle.dump(word_indices , f)

    with open('indices_word.pickle', 'wb') as g :
        pickle.dump(indices_word , g)

    #単語ファイルセーブ
    with open('words.pickle', 'wb') as h :
        pickle.dump(words , h)
        print(len(words))

    #コーパスセーブ
    with open('mat_urtext.pickle', 'wb') as ff :
        pickle.dump(mat_urtext , ff)    

if __name__ == '__main__':
    import numpy as np
    import pickle
    import glob

    generate_mat()


ファイル数 = 1
0
len(mat) 2864
total words: 1095
words_unk: 0
new total words: 1096
(2864, 1)
1096


In [94]:
with open('words.pickle', 'rb') as f :
        words=pickle.load(f)    
        print(len(words),words)

    #作成した辞書をロード
with open('word_indices.pickle', 'rb') as f :
    word_indices=pickle.load(f)

with open('indices_word.pickle', 'rb') as g :
    indices_word = pickle.load(g)

    #コーパスロード
with open('mat_urtext.pickle', 'rb') as ff :
    mat_urtext = pickle.load(ff)  

1096 ['\t', '!!', '%', '(', ')', '.5', '.\\', '/.', '0', '0310', '1', '10', '100', '2', '27', '3', '4', '5643648175', '6', '9', 'ADA', 'BGM', 'FF', 'GoTo', 'HP', 'M', 'NHK', 'REQREQ', 'RESRES', 'S', 'SmartNews', 'Yeaaaaahhhhhhhhh', 'You', 'YouTube', '^', 'anima', 'code', 'nhk', 'w', 'ww', 'www', 'wwww', '~', '×', 'Φ', 'ω', '…', '……', '①', '②', '③', '▼', '◎', '♡', '♪', '\u3000', '、', '。', '「', '」', '【', '】', '〜〜', 'あ', 'あがって', 'あけて', 'あって', 'あと', 'あなた', 'あの', 'あまり', 'あめ', 'あり', 'ありがとう', 'ある', 'あれまぁ', 'あんた', 'あー', 'い', 'いい', 'いう', 'いき', 'いし', 'いそう', 'いた', 'いって', 'いつ', 'いつまでも', 'いつも', 'いな', 'いはい', 'いやー', 'いらっと', 'いる', 'う', 'うまい', 'うんち', 'え', 'ええ', 'お', 'おいて', 'おじさん', 'おったら', 'おつ', 'おばさん', 'おら', 'お出かけ', 'お前', 'お構い', 'お陰', 'か', 'かけて', 'かして', 'かず', 'かな', 'かなり', 'かに', 'から', 'かわ', 'かわいい', 'かわいいー', 'かわいそう', 'が', 'がっかり', 'きた', 'きて', 'きめて', 'きれいで', 'く', 'くそ', 'ください', 'くらい', 'くる', 'くれて', 'くれる', 'くん', 'けぇ', 'けど', 'けん', 'こ', 'こい', 'こいつ', 'ここ', 'こじらせ', 'こそ', 'こと', 'この', 'こま', 'これ', 'こんな', 'こんにちはー', '